In [1]:
test1 = "o/ 근데 (70%)/(칠십 퍼센트)가 커 보이긴 하는데 (200)/(이백) 벌다 (140)/(백 사십) 벌면 빡셀걸? b/"
test2 = "근데 (3학년)/(삼 학년) 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?"

def bracket_filter(sentence):
  new_sentence = str()
  flag = False
  
  for ch in sentence:
    if ch == '(' and flag == False:
      flag = True
      continue
    if ch == '(' and flag == True:
      flag = False
      continue
    if ch != ')' and flag == False:
      new_sentence += ch
  return new_sentence

print(bracket_filter(test1))
print(bracket_filter(test2))


o/ 근데 칠십 퍼센트가 커 보이긴 하는데 이백 벌다 백 사십 벌면 빡셀걸? b/
근데 삼 학년 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?


In [2]:
import re

test1 = "o/ 근데 칠십 퍼센트가 커 보이긴 하는데 이백 벌다 백 사십 벌면 빡셀걸? b/"
test2 = "c# 배워봤어?"

def special_filter(sentence):
  SENTENCE_MARK = ['?', '!']
  NOISE = ['o', 'n', 'u', 'b', 'l']
  EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', '.', ',']
  
  new_sentence = str()
  for idx, ch in enumerate(sentence):
    if ch not in SENTENCE_MARK:
      # o/, n/ 등 처리
      if idx + 1 < len(sentence) and ch in NOISE and sentence[idx+1] == '/': 
        continue 

    if ch == '#': 
      new_sentence += '샾'

    elif ch not in EXCEPT: 
      new_sentence += ch

  pattern = re.compile(r'\s\s+')
  new_sentence = re.sub(pattern, ' ', new_sentence.strip())
  return new_sentence

print(special_filter(test1))
print(special_filter(test2))


근데 칠십 퍼센트가 커 보이긴 하는데 이백 벌다 백 사십 벌면 빡셀걸?
c샾 배워봤어?


In [3]:
# test = "o/ 근데 (70%)/(칠십 퍼센트)가 커 보이긴 하는데 (200)/(이백) 벌다 (140)/(백 사십) 벌면 빡셀걸? b/"
test = 'b/ 아 나 (3DS)/(쓰리 DS) 갖고 싶다. (3DS)/(쓰리 DS)'

def sentence_filter(raw_sentence):
  return special_filter(bracket_filter(raw_sentence))

print(sentence_filter(test))


아 나 쓰리 DS 갖고 싶다 쓰리 DS


In [4]:
chosung = ("ㄱ", "ㄲ", "ㄴ", "ㄷ", "ㄸ", "ㄹ", "ㅁ", "ㅂ", "ㅃ", "ㅅ", "ㅆ", "ㅇ", "ㅈ", "ㅉ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ")

jungsung = ("ㅏ", "ㅐ", "ㅑ", "ㅒ", "ㅓ", "ㅔ", "ㅕ", "ㅖ", "ㅗ", "ㅘ", "ㅙ", "ㅚ", "ㅛ", "ㅜ", "ㅝ", "ㅞ", "ㅟ", "ㅠ", "ㅡ", "ㅢ", "ㅣ")

jongsung = ("", "ㄱ", "ㄲ", "ㄳ", "ㄴ", "ㄵ", "ㄶ", "ㄷ", "ㄹ", "ㄺ", "ㄻ", "ㄼ", "ㄽ", "ㄾ", "ㄿ", "ㅀ", "ㅁ", "ㅂ", "ㅄ", "ㅅ", "ㅆ", "ㅇ", "ㅈ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ")

def isHangeul(one_character):
    return 0xAC00 <= ord(one_character[:1]) <= 0xD7A3

def hangeulExplode(one_hangeul):
    a = one_hangeul[:1]
    if isHangeul(a) != True:
        return False
    b = ord(a) - 0xAC00
    cho = b // (21*28)
    jung = b % (21*28) // 28
    jong = b % 28
    if jong == 0:
        return (chosung[cho], jungsung[jung])
    else:
        return (chosung[cho], jungsung[jung], jongsung[jong])

def hangeulJoin(inputlist):
    result = ""
    cho, jung, jong = 0, 0, 0
    inputlist.insert(0, "")
    while len(inputlist) > 1:
        if inputlist[-1] in jongsung:
            if inputlist[-2] in jungsung:
                jong = jongsung.index(inputlist.pop())
            
            else:
                result += inputlist.pop()
        elif inputlist[-1] in jungsung:
            if inputlist[-2] in chosung:
                jung = jungsung.index(inputlist.pop())
                cho = chosung.index(inputlist.pop())
                result += chr(0xAC00 + ((cho*21)+jung)*28+jong)
                cho, jung, jong = 0, 0, 0
            else:
                result += inputlist.pop()

        else:
            result += inputlist.pop()
    else:
        return result[::-1]

def pureosseugi(inputtext):
    result = ""
    for i in inputtext:
        if isHangeul(i) == True:
            for j in hangeulExplode(i):
                result += j
        else:
            result += i
    
    return result

def moasseugi(inputtext):
    t1 = []
    for i in inputtext:
        t1.append(i)

    return hangeulJoin(t1)


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

class Phoneme:
  def __init__ (self):
    self.BLANK_INDEX = 0
    self.START_INDEX = 1
    self.END_INDEX = 2
    self.SPACE_INDEX = 3
    self.OOV_INDEX = 4

class PhonemeGenerator (Phoneme):
  def __init__ (self, labels):
    super(PhonemeGenerator, self).__init__()
    self.phonemes = ['<BLANK>', '<START>', '<END>', ' ']

    texts = []
    for label in labels:
      for phoneme in label:
        if not isHangeul(phoneme) and not phoneme == ' ' and not phoneme == '?':
          break
        for pure in pureosseugi(phoneme):
          texts.append(pure)

    tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')
    tokenizer.fit_on_texts(texts)

    for i in range(1, len(tokenizer.index_word) + 1):
      self.phonemes.append(tokenizer.index_word[i])

    self.phoneme_index = {}
    for i, phoneme in enumerate(self.phonemes):
      self.phoneme_index[phoneme] = i
    
    self.VOCAB_SIZE = len(self.phonemes)
  
  def save (self, filename):
    with open(filename, 'wb') as f:
      pickle.dump((self.phonemes, self.phoneme_index), f)


class PhonemeLoader (Phoneme):
  def __init__ (self, filename):
    super(PhonemeLoader, self).__init__()
    with open(filename, 'rb') as f:
      self.phonemes, self.phoneme_index = pickle.load(f)

    self.VOCAB_SIZE = len(self.phonemes)


In [6]:
import random
import os
import numpy as np
import librosa

class Dataset:
  def get_data (self, idx):
    return (
      self.extract_spectrogram(self.file_phoneme_vector_pairs[idx][0]),
      [0] + self.file_phoneme_vector_pairs[idx][1]
    )
  
  def get_len (self):
    return len(self.file_phoneme_vector_pairs)

  def extract_spectrogram (self, filename):
    waveform = np.memmap(filename, dtype='h', mode='r')

    sr = 16000
    st = 1 / sr

    t = np.linspace(0, waveform.shape[0] * st, waveform.shape[0])

    S = librosa.feature.melspectrogram(y=np.float32(waveform), n_fft=512, hop_length=256, window='hamming', sr=sr, n_mels=128)
    log_S = librosa.power_to_db(S, ref=np.max)
    log_S = np.transpose(log_S)

    return log_S
  
  def save (self, filename):
    with open(filename, 'wb') as f:
      pickle.dump((self.phoneme, self.file_phoneme_vector_pairs), f)

class RandomDataset (Dataset):
  def __init__ (self):
    super(RandomDataset, self).__init__()
    with open('./ko-audio-dataset/sentence-script/KsponSpeech_scripts/train.trn', 'r', encoding='utf-8') as file:
      lines = file.readlines()
    def process_line (line):
      line = line.split(' :: ')
      line[0] = os.path.join('./ko-audio-dataset/audio/', line[0])
      line[1] = sentence_filter(line[1])
      return line
    file_label_pairs = list(map(process_line, lines))
    file_label_pairs = list(filter(lambda pair: not self.check_skip_label(pair[1]), file_label_pairs))
    self.phoneme = PhonemeGenerator([label for file, label in file_label_pairs])
    file_phoneme_pairs = list(map(lambda pair: (pair[0], pureosseugi(pair[1])), file_label_pairs))
    self.file_phoneme_vector_pairs = [[pair[0], [self.phoneme.phoneme_index[phoneme] for phoneme in pair[1]]] for pair in file_phoneme_pairs]
    random.shuffle(self.file_phoneme_vector_pairs)
  
  def check_skip_hangul (self, hangul):
    return not isHangeul(hangul) and not hangul == ' ' and not hangul == '?'

  def check_skip_label (self, label):
    for hangul in label:
      if self.check_skip_hangul(hangul):
        return True
    return False

class LoadedDataset (Dataset):
  def __init__ (self, filename):
    super(LoadedDataset, self).__init__()
    with open(filename, 'rb') as f:
      self.phoneme, self.file_phoneme_vector_pairs = pickle.load(f)



c:\Users\akswnd98\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [7]:
dataset = RandomDataset()
dataset.save('dataset2.pickle')

In [8]:
dataset = LoadedDataset('dataset2.pickle')

In [9]:
class DatasetSizeLimitingIterator:
  def __init__ (self, dataset, limit, idx):
    self.dataset = dataset
    self.limit = limit
    self.idx = idx
    while self.idx < self.dataset.get_len():
      data = self.dataset.get_data(self.idx)
      if data[0].shape[0] * len(data[1]) * len(self.dataset.phoneme.phonemes) <= self.limit:
        break
      self.idx += 1

  def next (self):
    idx = self.idx + 1
    while idx < self.dataset.get_len():
      data = self.dataset.get_data(idx)
      if data[0].shape[0] * len(data[1]) * len(self.dataset.phoneme.phonemes) <= self.limit:
        break
      idx += 1
    return DatasetSizeLimitingIterator(self.dataset, self.limit, idx)
  
  def check_end (self):
    return self.idx >= self.dataset.get_len()

  def get_value (self):
    return self.dataset.get_data(self.idx)

  def save (self, filename):
    with open(filename, 'wb') as f:
      pickle.dump((self.limit, self.idx), f)

class IteratorLoader:
  @staticmethod
  def load (dataset, filename):
    with open(filename, 'rb') as f:
      limit, idx = pickle.load(f)
    return DatasetSizeLimitingIterator(dataset, limit, idx)


In [10]:
iterator = DatasetSizeLimitingIterator(dataset, 100000, 0)
iterator.save('iterator2.pickle')

In [11]:
iterator = IteratorLoader.load(dataset, 'iterator2.pickle')

In [12]:
import os

class DatasetStepExtractorLoader:
  @staticmethod
  def load (path):
    dataset = LoadedDataset(os.path.join(path, 'dataset.pickle'))
    return SavableDatasetStepExtractor(
      dataset,
      pickle.load(open(os.path.join(path, 'step_size.pickle'), 'rb')),
      IteratorLoader.load(dataset, os.path.join(path, 'iterator.pickle'))
    )

class DatasetStepExtractor:
  def __init__ (self, dataset, step_size, iterator):
    self.dataset = dataset
    self.step_size = step_size
    self.value = []
    self.iterator = iterator
    while not iterator.check_end() and len(self.value) < self.step_size:
      self.value.append(iterator.get_value())
      iterator = iterator.next()
    self.is_end = len(self.value) < self.step_size
    self.next_iterator = iterator
  
  def check_end (self):
    return self.is_end
  
  def get_value (self):
    return self.value
  
  def get_next_extractor (self):
    return DatasetStepExtractor(self.dataset, self.step_size, self.next_iterator)

class SavableDatasetStepExtractor (DatasetStepExtractor):
  def save (self, saver):
    saver.save(self.dataset, self.step_size, self.iterator)

class DatasetStepExtractorSaver:
  def __init__ (self, path):
    self.path = path
  
  def save (self, dataset, step_size, iterator):
    os.makedirs(self.path, exist_ok=True)

class IteratorSaver (DatasetStepExtractorSaver):
  def save (self, dataset, step_size, iterator):
    super().save(dataset, step_size, iterator)
    iterator.save(os.path.join(self.path, 'iterator.pickle'))
  
class DatasetSaver (DatasetStepExtractorSaver):
  def save (self, dataset, step_size, iterator):
    super().save(dataset, step_size, iterator)
    dataset.save(os.path.join(self.path, 'dataset.pickle'))

class StepSizeSaver (DatasetStepExtractorSaver):
  def save (self, dataset, step_size, iterator):
    super().save(dataset, step_size, iterator)
    with open(os.path.join(self.path, 'step_size.pickle'), 'wb') as f:
      pickle.dump(step_size, f)

class DatasetStepExtractorSaver (DatasetStepExtractorSaver):
  def save (self, dataset, step_size, iterator):
    super().save(dataset, step_size, iterator)
    iterator.save(os.path.join(self.path, 'iterator.pickle'))
    dataset.save(os.path.join(self.path, 'dataset.pickle'))
    with open(os.path.join(self.path, 'step_size.pickle'), 'wb') as f:
      pickle.dump(step_size, f)


In [13]:
extractor = SavableDatasetStepExtractor(dataset, 100, iterator)
extractor.save(DatasetStepExtractorSaver('test_extractor2'))

In [14]:
extractor = DatasetStepExtractorLoader.load('test_extractor2')

In [15]:
BATCH_SIZE = 1
BATCH_CNT = 1
STEP_SIZE = BATCH_SIZE * BATCH_CNT
SIZE_LIMIT = 2000000
extractor = SavableDatasetStepExtractor(dataset, STEP_SIZE, DatasetSizeLimitingIterator(dataset, SIZE_LIMIT, 0))
extractor.save(DatasetStepExtractorSaver('extractor2'))


In [16]:
# extractor = DatasetStepExtractorLoader.load('extractor2')

In [17]:
################ 여기 까지 dataset 부분 ###############

In [18]:
import tensorflow as tf
import numpy as np

In [19]:
def rnnt_loss (logits, labels, time_lengths, label_lengths):
  log_pr = tf.math.log_softmax(logits, axis=-1)
  pr = pr_loss(log_pr, labels, time_lengths, label_lengths)
  ret = tf.reduce_sum(pr)
  return ret

@tf.custom_gradient
def pr_loss (log_pr, labels, time_lengths, label_lengths):
  LOG_0 = float('-inf')
  batch_size = log_pr.shape[0]
  max_time_lengths = log_pr.shape[1]
  max_label_lengths = log_pr.shape[2]

  def get_truth_log_pr (log_pr, labels):
    labels_one_hot = tf.one_hot(labels[:, 1: ], tf.shape(log_pr)[-1], axis=-1, dtype=tf.float32)
    labels_one_hot = tf.expand_dims(labels_one_hot, axis=1)
    labels_one_hot = tf.repeat(labels_one_hot, tf.shape(log_pr)[1], axis=1)
    ret = tf.reduce_sum(log_pr[:, :, : -1, :] * labels_one_hot, axis=-1)
    ret = tf.concat([ret, LOG_0 * tf.ones((tf.shape(log_pr)[0], tf.shape(log_pr)[1], 1))], axis = -1)
    return ret
  
  def get_blank_log_pr (log_pr):
    return log_pr[:, :, :, 0]

  truth_log_pr = get_truth_log_pr(log_pr, labels)
  blank_log_pr = get_blank_log_pr(log_pr)

  def get_alpha (truth_log_pr, blank_log_pr):
    reversed_truth_log_pr = tf.reverse(truth_log_pr, axis=[-2])
    padded_truth_log_pr = tf.pad(
      reversed_truth_log_pr,
      [[0, 0], [0, 0], [tf.shape(reversed_truth_log_pr)[-2] - 1, 0]],
      constant_values=LOG_0
    )
    truth_diag = tf.linalg.diag_part(
      padded_truth_log_pr,
      k=(0, tf.shape(padded_truth_log_pr)[-1] - 1),
      padding_value=LOG_0,
      align='LEFT_RIGHT'
    )
    truth_diag = tf.reverse(truth_diag, axis=[-2])
    truth_diag = tf.pad(truth_diag, [[0, 0], [0, 0], [1, 0]], constant_values=LOG_0)

    reversed_blank_log_pr = tf.reverse(blank_log_pr, axis=[-2])
    padded_blank_log_pr = tf.pad(
      reversed_blank_log_pr,
      [[0, 0], [0, 0], [tf.shape(reversed_blank_log_pr)[-2] - 1, 0]],
      constant_values=LOG_0
    )
    blank_diag = tf.linalg.diag_part(
      padded_blank_log_pr,
      k=(0, tf.shape(padded_blank_log_pr)[-1] - 1),
      padding_value=LOG_0,
      align='LEFT_RIGHT'
    )
    blank_diag = tf.reverse(blank_diag, axis=[-2])
    blank_diag = tf.pad(blank_diag, [[0, 0], [0, 0], [0, 1]], constant_values=LOG_0)

    initial_diag = tf.concat([
      LOG_0 * tf.ones((tf.shape(blank_diag)[0], tf.shape(blank_diag)[-1] - 2), tf.float32),
      tf.zeros((tf.shape(blank_diag)[0], 1), dtype=tf.float32)
    ], axis=-1)

    def step (a, x):
      t, b = x
      return (
        tf.reduce_logsumexp(
          tf.stack([
            tf.concat([LOG_0 * tf.ones((tf.shape(a)[0], 1), dtype=tf.float32), a], axis=-1) + t,
            tf.concat([a, LOG_0 * tf.ones((tf.shape(a)[0], 1), dtype=tf.float32)], axis=-1) + b
          ]), axis=0
        )[:, 1: ]
      )

    alpha_diag = tf.concat([
      tf.expand_dims(initial_diag, axis=0),
      tf.scan(step, (tf.transpose(truth_diag, perm=[1, 0, 2]), tf.transpose(blank_diag, perm=[1, 0, 2])), initial_diag)
    ], axis=0)

    alpha_diag = tf.transpose(alpha_diag, perm=[1, 0, 2])

    alpha_diag = tf.reverse(alpha_diag, axis=[-1])

    alpha_diag = tf.transpose(alpha_diag, perm=[0, 2, 1])
    alpha = tf.linalg.diag_part(alpha_diag, k=(0, max_label_lengths - 1))
    alpha = tf.reshape(alpha, (tf.shape(truth_log_pr)[0], tf.shape(truth_log_pr)[2], tf.shape(truth_log_pr)[1]))
    alpha = tf.reverse(alpha, axis=[-2])
    alpha = tf.transpose(alpha, perm=[0, 2, 1])
    return alpha

  def get_beta (truth_log_pr, blank_log_pr):
    reversed_truth_log_pr = tf.reverse(truth_log_pr, axis=[-2])
    reversed_truth_log_pr = tf.concat([
      reversed_truth_log_pr[:, :, : -1],
      LOG_0 * tf.ones((tf.shape(reversed_truth_log_pr)[0], tf.shape(reversed_truth_log_pr)[-2], 1), dtype=tf.float32)
    ], axis=-1)
    padded_truth_log_pr = tf.pad(
      reversed_truth_log_pr,
      [[0, 0], [0, 0], [tf.shape(reversed_truth_log_pr)[-2] - 1, 0]],
      constant_values=LOG_0
    )
    truth_diag = tf.linalg.diag_part(
      padded_truth_log_pr,
      k=(0, tf.shape(padded_truth_log_pr)[-1] - 1),
      padding_value=LOG_0,
      align='LEFT_RIGHT'
    )
    truth_diag = tf.transpose(truth_diag, perm=[1, 0, 2])

    reversed_blank_log_pr = tf.reverse(blank_log_pr, axis=[-2])
    reversed_blank_log_pr = tf.concat([
      LOG_0 * tf.ones((tf.shape(reversed_blank_log_pr)[0], 1, tf.shape(reversed_blank_log_pr)[-1]), dtype=tf.float32),
      reversed_blank_log_pr[:, 1:, :]
    ], axis=-2)
    padded_blank_log_pr = tf.pad(
      reversed_blank_log_pr,
      [[0, 0], [0, 0], [tf.shape(reversed_blank_log_pr)[-2] - 1, 0]],
      constant_values=LOG_0
    )
    blank_diag = tf.linalg.diag_part(
      padded_blank_log_pr,
      k=(0, tf.shape(padded_blank_log_pr)[-1] - 1),
      padding_value=LOG_0,
      align='LEFT_RIGHT'
    )
    blank_diag = tf.transpose(blank_diag, perm=[1, 0, 2])

    initial_diag = tf.concat([
      tf.reshape(blank_log_pr[:, -1, -1], (tf.shape(blank_log_pr)[0], 1)),
      # -0.5 * tf.ones((tf.shape(blank_diag)[0], 1), dtype=tf.float32),
      LOG_0 * tf.ones((tf.shape(blank_log_pr)[0], tf.shape(blank_log_pr)[-2] - 1), tf.float32)
    ], axis=-1)

    def step (a, x):
      t, b = x
      return (
        tf.reduce_logsumexp(
          tf.stack([
            a + t,
            tf.concat([LOG_0 * tf.ones((tf.shape(a)[0], 1), dtype=tf.float32), a[:, : -1]], axis=-1) + b
          ]), axis=0
        )
      )

    beta_diag = tf.concat([
      tf.expand_dims(initial_diag, axis=0),
      tf.scan(step, (truth_diag[1: ], blank_diag[1: ]), initial_diag)
    ], axis=0)

    beta_diag = tf.transpose(beta_diag, [1, 0, 2])

    beta = tf.linalg.diag_part(
      beta_diag,
      k=(-max_label_lengths + 1, 0),
      padding_value=LOG_0,
      align='LEFT_RIGHT'
    )
    beta = tf.reshape(beta, (tf.shape(truth_log_pr)[0], tf.shape(truth_log_pr)[2], tf.shape(truth_log_pr)[1]))
    beta = tf.transpose(beta, [0, 2, 1])
    beta = tf.reverse(beta, axis=[1, 2])
    return beta

  alpha = get_alpha(truth_log_pr, blank_log_pr)
  beta = get_beta(truth_log_pr, blank_log_pr)

  total_log_pr = beta[:, 0, 0]

  def grad (upstream):
    blank_grads = \
      alpha + tf.concat([
        beta[:, 1:, :],
        tf.pad(LOG_0 * tf.ones((tf.shape(beta)[0], 1, tf.shape(beta)[-1] - 1), dtype=tf.float32), [[0, 0], [0, 0], [0, 1]], constant_values=0)
      ], axis=1) \
      - tf.reshape(total_log_pr, shape=(tf.shape(total_log_pr)[0], 1, 1))
    truth_grads = \
      alpha + tf.pad(beta[:, :, 1: ], [[0, 0], [0, 0], [0, 1]], constant_values=LOG_0) \
      - tf.reshape(total_log_pr, shape=(tf.shape(total_log_pr)[0], 1, 1))
    blank_one_hot = tf.one_hot(tf.zeros_like(labels, dtype=tf.int32), tf.shape(log_pr)[-1], dtype=tf.float32)
    blank_one_hot = tf.expand_dims(blank_one_hot, axis=1)
    blank_one_hot = tf.repeat(blank_one_hot, tf.shape(log_pr)[1], axis=1)
    truth_one_hot = tf.one_hot(labels[:, 1: ], tf.shape(log_pr)[-1], dtype=tf.float32)
    truth_one_hot = tf.concat([truth_one_hot, tf.zeros((tf.shape(log_pr)[0], 1, tf.shape(log_pr)[-1]), dtype=tf.float32)], axis=-2)
    truth_one_hot = tf.expand_dims(truth_one_hot, axis=1)
    truth_one_hot = tf.repeat(truth_one_hot, tf.shape(log_pr)[1], axis=1)
    return (
      [
        tf.reshape(-upstream, shape=(tf.shape(upstream)[0], 1, 1))
        * (
          tf.exp(tf.expand_dims(blank_grads, axis=-1) + log_pr) * blank_one_hot
          + tf.exp(tf.expand_dims(truth_grads, axis=-1) + log_pr) * truth_one_hot
        )
      ] +
      [None] * 3
    )

  return -total_log_pr, grad


In [20]:
EMBEDDING_DIM = 256
UNITS = 1024
CODER_OUTPUT_DIM = 512
JOINT_NET_INNER_DIM = 1024

In [21]:
class Encoder (tf.keras.Model):
  def __init__ (self, units, output_dim):
    super(Encoder, self).__init__(self)
    self.gru = tf.keras.layers.GRU(units, recurrent_initializer='glorot_uniform', return_sequences=True, return_state=True)
    self.fc = tf.keras.layers.Dense(output_dim)
    
  def call (self, inputs):
    rnn_outputs, hidden_states = self.gru(inputs)
    outputs = self.fc(rnn_outputs)
    return outputs


In [22]:
class Decoder (tf.keras.Model):
  def __init__ (self, vocab_size, embedding_dim, units, output_dim):
    super(Decoder, self).__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(units, recurrent_initializer='glorot_uniform', return_sequences=True, return_state=True)
    self.fc = tf.keras.layers.Dense(output_dim)

  def call (self, inputs):
    embedded = self.embedding(inputs)
    outputs, hidden_states = self.gru(embedded)
    outputs = self.fc(outputs)
    return outputs, hidden_states

In [23]:
class JointNet (tf.keras.Model):
  def __init__ (self, inner_dim, vocab_size):
    super(JointNet, self).__init__()
    self.forward_layer = tf.keras.layers.Dense(inner_dim, activation='tanh')
    self.project_layer = tf.keras.layers.Dense(vocab_size)
  
  def call (self, inputs):
    enc_outputs, dec_outputs = inputs
    joint_inputs = tf.expand_dims(enc_outputs, axis=2) + tf.expand_dims(dec_outputs, axis=1)
    outputs = self.forward_layer(joint_inputs)
    outputs = self.project_layer(outputs)
    return outputs


In [24]:
class Transducer (tf.keras.Model):
  def __init__ (self, encoder, decoder, joint_net):
    super(Transducer, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.joint_net = joint_net

  def __init__ (self, embedding_dim, units, coder_output_dim, joint_net_inner_dim, vocab_size):
    super(Transducer, self).__init__()
    self.encoder = Encoder(units, coder_output_dim)
    self.decoder = Decoder(vocab_size, embedding_dim, units, coder_output_dim)
    self.joint_net = JointNet(joint_net_inner_dim, vocab_size)
    self.vocab_size = vocab_size
  
  def call (self, inputs):
    enc_inputs, dec_inputs = inputs
    enc_outputs = self.encoder(enc_inputs)
    dec_outputs, dec_states = self.decoder(dec_inputs)
    outputs = self.joint_net([enc_outputs, dec_outputs])
    return outputs


In [25]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import numpy as np
import librosa

class Trainer:
  def __init__ (self, transducer, optimizer, batch_size, dataset_extractor, save_time_delta):
    self.transducer = transducer
    self.optimizer = optimizer
    self.batch_size = batch_size
    self.dataset_extractor = dataset_extractor
    self.save_time_delta = save_time_delta

  def train_step (self):
    datas = self.dataset_extractor.get_value()
    acc_grads = [tf.zeros_like(var) for var in self.transducer.trainable_variables]
    acc_loss = 0
    for i in range(0, len(datas), self.batch_size):
      batch_datas = datas[i: i + self.batch_size]
      label_lengths = self.get_label_lengths(batch_datas)
      label_lengths = tf.convert_to_tensor(label_lengths, dtype=tf.int32)
      labels = pad_sequences(list(map(lambda x: x[1], batch_datas)), maxlen=self.get_labels_maxlen(batch_datas), truncating='post', padding='post')
      labels = list(map(lambda x: x[1], batch_datas))
      labels = tf.constant(labels, dtype=tf.int32)
      spectrograms = [x[0] for x in batch_datas]
      spectrogram_lengths = self.get_spectrogram_lengths(spectrograms)
      spectrogram_lengths = tf.convert_to_tensor(spectrogram_lengths, dtype=tf.int32)
      spectrograms = tf.cast(pad_sequences(spectrograms, maxlen=self.get_spectrograms_maxlen(spectrograms), truncating='post', padding='post'), dtype=tf.float32)
      with tf.GradientTape() as tape:
        logits = self.transducer([spectrograms, labels])
        loss = rnnt_loss(
          logits,
          labels,
          spectrogram_lengths,
          label_lengths
        )
      acc_loss += loss.numpy()
      grads = tape.gradient(loss, self.transducer.trainable_variables)
      acc_grads = [acc_grad + grad for acc_grad, grad in zip(acc_grads, grads)]
    acc_grads = [acc_grad / self.dataset_extractor.step_size for acc_grad in acc_grads]
    self.optimizer.apply_gradients(zip(acc_grads, self.transducer.trainable_variables))
    self.dataset_extractor = self.dataset_extractor.get_next_extractor()
    extractor.save(IteratorSaver('extractor2'))
    
    return acc_loss / self.dataset_extractor.step_size

  def train (self):
    loss = 0
    cnt = 0
    prev_save_time = 0
    cur_block_size = 0
    while not self.dataset_extractor.check_end():
      # loss = self.train_step()
      # cnt += self.dataset_extractor.step_size
      loss += self.train_step()
      cnt += self.dataset_extractor.step_size
      if cnt % 100 == 0:
        print('cnt: {}'.format(cnt))
        print('loss: {}'.format(loss / 100))
        loss = 0
      if cnt >= prev_save_time + self.save_time_delta:
        self.transducer.save_weights('./checkpoints/no{}/ckpt'.format(cnt))
        prev_save_time = cnt

  def get_labels_maxlen (self, batch_datas):
    ret = 0
    for file_label_vector_pair in batch_datas:
      ret = max(ret, len(file_label_vector_pair[1]))
    return ret
  
  def get_label_lengths (self, batch_datas):
    ret = list(map(lambda x: len(x[1]), batch_datas))
    return ret
  
  def get_spectrograms_maxlen (self, spectrograms):
    ret = 0
    for spectrogram in spectrograms:
      ret = max(ret, spectrogram.shape[0])
    return ret

  def get_spectrogram_lengths (self, spectrograms):
    ret = list(map(lambda x: x.shape[0], spectrograms))
    return ret
      

In [26]:
VOCAB_SIZE = dataset.phoneme.VOCAB_SIZE
SAVE_TIME_DELTA = 5000


In [27]:
# encoder = Encoder(UNITS, CODER_OUTPUT_DIM)
# decoder = Decoder(dataset.phoneme.VOCAB_SIZE, EMBEDDING_DIM, UNITS, CODER_OUTPUT_DIM)
# joint_net = JointNet(1024, dataset.phoneme.VOCAB_SIZE)
# transducer = Transducer(encoder, decoder, joint_net)

transducer = Transducer(EMBEDDING_DIM, UNITS, CODER_OUTPUT_DIM, JOINT_NET_INNER_DIM, dataset.phoneme.VOCAB_SIZE)

transducer([
  tf.keras.Input(shape=(None, 128), dtype=tf.float32),
  tf.keras.Input(shape=(None, ), dtype=tf.int32)
])

transducer.summary()

# transducer = Transducer(EMBEDDING_DIM, UNITS, CODER_OUTPUT_DIM, JOINT_NET_INNER_DIM, VOCAB_SIZE)
# transducer.load_weights('./checkpoints/no5000/ckpt')

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
trainer = Trainer(transducer, optimizer, BATCH_SIZE, extractor, SAVE_TIME_DELTA)

with tf.device('/gpu:0'):
  trainer.train()


Model: "transducer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  4069888   
                                                                 
 decoder (Decoder)           multiple                  4477440   
                                                                 
 joint_net (JointNet)        multiple                  582712    
                                                                 
Total params: 9,130,040
Trainable params: 9,130,040
Non-trainable params: 0
_________________________________________________________________
cnt: 100
loss: 149.0256834411621
cnt: 200
loss: 127.6614616394043
cnt: 300
loss: 142.0180096912384
cnt: 400
loss: 127.00405046463013
cnt: 500
loss: 126.21211320400238
cnt: 600
loss: 121.41114798545837
cnt: 700
loss: 111.66791275978089
cnt: 800
loss: 118.10227319717407
cnt: 900
loss: 105.90308545351029
cnt: 10

KeyboardInterrupt: 

In [ ]:
spectrogram, labels = extractor.get_value()[0]

logits = transducer((tf.convert_to_tensor([spectrogram]), tf.convert_to_tensor([labels])))
print(tf.argmax(logits, axis=-1)[:, :, : ][0])

In [ ]:
class AudioDecoder:
  def __init__ (self, transducer):
    self.transducer = transducer
  
  def decode (self, spectrogram):
    label = [0]
    for i in range(spectrogram.shape[0]):
      sub_spectrogram = spectrogram[0: i + 1]
      sub_spectrogram = tf.convert_to_tensor(sub_spectrogram)
      sub_spectrogram = tf.expand_dims(sub_spectrogram, axis=0)
      sub_label = tf.constant([label], dtype=tf.int32)
      sub_rst = self.transducer((sub_spectrogram, sub_label))
      sub_rst = tf.argmax(sub_rst[0][-1][-1], axis=-1).numpy()
      label.append(sub_rst)
    return label


In [ ]:
decoder = AudioDecoder(transducer)

In [ ]:
a = list(filter(lambda x: x != 0, decoder.decode(spectrogram)))
print(moasseugi(vector_to_sentence([0] + a)))


In [ ]:
print(labels)

In [ ]:
def vector_to_sentence (x):
  return ''.join([dataset.phoneme.phonemes[idx] for idx in x][1: ])


In [ ]:
print(moasseugi(vector_to_sentence(labels)))

In [ ]:
rst = []
x, y = 0, 0
while x < logits.shape[2] and y < logits.shape[1]:
  sel = tf.argmax(logits[0][y][x], axis=-1).numpy()
  rst.append(sel)
  if sel == 0:
    y += 1
  else:
    x += 1

print(moasseugi(vector_to_sentence([0] + list(filter(lambda x: x != 0, rst)))))